In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect("./chinook.db")

In [3]:
def sql_query(sql_command, con):
    return pd.read_sql_query(sql_command, con)

> ซ้อม SQL จากการสอบค่ายม่วง

## 
### INNER JOIN หลายชั้น
- อยากรู้ว่า customer ซื้อ track อะไร

In [41]:
sql_command = """
SELECT tracks.name AS track_name, customers.firstname AS customer_name, customers.country
FROM invoices
INNER JOIN customers ON customers.customerId = invoices.customerId
INNER JOIN invoice_items ON invoice_items.invoiceId = invoices.invoiceId
INNER JOIN tracks ON tracks.trackId = invoice_items.trackId
"""

sql_query(sql_command, con)

,track_name,customer_name,Country
0,Experiment In Terra,Luís,Brazil
1,Take the Celestra,Luís,Brazil
2,Shout It Out Loud,Luís,Brazil
3,Calling Dr. Love,Luís,Brazil
4,Strutter,Luís,Brazil
...,...,...,...
2235,Esporrei Na Manivela,Puja,India
2236,No Fundo Do Quintal Da Escola,Puja,India
2237,Que Luz É Essa,Puja,India
2238,The Power Of Equality,Puja,India


- แปลงผลลัพธ์ใน column ที่ต้องการ เลือกเฉพาะผลลัพธ์ 

In [26]:
sql_command = """
SELECT tracks.name AS track_name, customers.firstname AS customer_name,
    CASE customers.country
        WHEN "USA" THEN "Domestic"
        ELSE "Foreign"
    END country_group
FROM invoices
INNER JOIN customers ON customers.customerId = invoices.customerId
INNER JOIN invoice_items ON invoice_items.invoiceId = invoices.invoiceId
INNER JOIN tracks ON tracks.trackId = invoice_items.trackId
WHERE country_group = "Domestic"
"""

sql_query(sql_command, con)

,track_name,customer_name,country_group
0,Valentino's,Frank,Domestic
1,Promises,Frank,Domestic
2,Signe,Frank,Domestic
3,Ghost Of The Navigator,Frank,Domestic
4,Wildest Dreams,Frank,Domestic
...,...,...,...
489,"Battlestar Galactica, Pt. 1",Julia,Domestic
490,"Lost Planet of the Gods, Pt. 2",Julia,Domestic
491,"The Gun On Ice Planet Zero, Pt. 2",Julia,Domestic
492,"The Living Legend, Pt. 2",Julia,Domestic


- นับว่า customer ซื้อไปกี่เพลง

In [47]:
sql_command = """
SELECT COUNT(tracks.name) AS count_track_name, customers.firstname AS customer_name, customers.country
FROM invoices
INNER JOIN customers ON customers.customerId = invoices.customerId
INNER JOIN invoice_items ON invoice_items.invoiceId = invoices.invoiceId
INNER JOIN tracks ON tracks.trackId = invoice_items.trackId
GROUP BY customer_name
ORDER BY count_track_name DESC, country
"""

sql_query(sql_command, con).head(15)

,count_track_name,customer_name,Country
0,76,Mark,Canada
1,76,Frank,USA
2,38,Diego,Argentina
3,38,Astrid,Austria
4,38,Daan,Belgium
5,38,Alexandre,Brazil
6,38,Eduardo,Brazil
7,38,Fernanda,Brazil
8,38,Luís,Brazil
9,38,Roberto,Brazil


- ใครที่ซื้อเพลงที่ต้องการ (IN+subquery)

In [48]:
sql_command = """
SELECT tracks.name AS track_name, customers.firstname AS customer_name, customers.country
FROM invoices
INNER JOIN customers ON customers.customerId = invoices.customerId
INNER JOIN invoice_items ON invoice_items.invoiceId = invoices.invoiceId
INNER JOIN tracks ON tracks.trackId = invoice_items.trackId
WHERE track_name IN (
    SELECT name
    FROM tracks
    WHERE trackid < 100
)
"""

sql_query(sql_command, con)

,track_name,customer_name,Country
0,Balls to the Wall,Leonie,Germany
1,Restless and Wild,Leonie,Germany
2,"Canta, Canta Mais",François,Canada
3,Cochise,François,Canada
4,Hypnotize,François,Canada
...,...,...,...
64,Forgiven,Phil,United Kingdom
65,Wake Up,Phil,United Kingdom
66,I Can't Remember,Phil,United Kingdom
67,I Know Somethin (Bout You),Phil,United Kingdom


## 
### SUM/AVG/COUNT

- เพลง/แนวเพลง

In [50]:
sql_command = """
SELECT tracks.trackId, tracks.Name, genres.name
FROM tracks
INNER JOIN genres ON genres.genreId = tracks.genreId
"""

sql_query(sql_command, con)

,TrackId,Name,Name
0,1,For Those About To Rock (We Salute You),Rock
1,2,Balls to the Wall,Rock
2,3,Fast As a Shark,Rock
3,4,Restless and Wild,Rock
4,5,Princess of the Dawn,Rock
...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Classical
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Classical
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Classical


- แต่ละแนวมีกี่เพลง เลือกเอาแนวที่มีเพลงมากกว่า ... เพลง

In [57]:
sql_command = """
SELECT COUNT(tracks.Name) AS count_track, genres.name
FROM tracks
INNER JOIN genres ON genres.genreId = tracks.genreId
GROUP BY genres.name
ORDER BY count_track DESC
"""

sql_query(sql_command, con).head(10)

,count_track,Name
0,1297,Rock
1,579,Latin
2,374,Metal
3,332,Alternative & Punk
4,130,Jazz
5,93,TV Shows
6,81,Blues
7,74,Classical
8,64,Drama
9,61,R&B/Soul


In [60]:
sql_command = """
SELECT COUNT(tracks.Name) AS count_track, genres.name
FROM tracks
INNER JOIN genres ON genres.genreId = tracks.genreId
GROUP BY genres.name
HAVING count_track > 50
ORDER BY count_track DESC
"""

sql_query(sql_command, con)

,count_track,Name
0,1297,Rock
1,579,Latin
2,374,Metal
3,332,Alternative & Punk
4,130,Jazz
5,93,TV Shows
6,81,Blues
7,74,Classical
8,64,Drama
9,61,R&B/Soul


- ราคาต่อ 1 invoice และ จำนวนรายการที่ซื้อ

In [67]:
sql_command = """
SELECT invoice_items.invoiceId,
    SUM(invoice_items.quantity) AS total_quantity,
    SUM(invoice_items.unitprice * invoice_items.quantity) AS total_price
FROM invoice_items
GROUP BY invoice_items.invoiceId
ORDER BY total_price DESC
"""

sql_query(sql_command, con)

,InvoiceId,total_quantity,total_price
0,404,14,25.86
1,299,14,23.86
2,96,14,21.86
3,194,14,21.86
4,201,14,18.86
...,...,...,...
407,377,1,0.99
408,384,1,0.99
409,391,1,0.99
410,398,1,0.99


- ราคาเฉลี่ยของเพลงแต่ละแนว

In [69]:
sql_command = """
SELECT AVG(tracks.unitPrice), genres.name
FROM tracks
INNER JOIN genres ON genres.genreId = tracks.genreId
GROUP BY tracks.genreId
"""

sql_query(sql_command, con)

,AVG(tracks.unitPrice),Name
0,0.99,Rock
1,0.99,Jazz
2,0.99,Metal
3,0.99,Alternative & Punk
4,0.99,Rock And Roll
5,0.99,Blues
6,0.99,Latin
7,0.99,Reggae
8,0.99,Pop
9,0.99,Soundtrack
